In [10]:
# Add python script files to the system env
import sys  
sys.path.insert(1, "..\\My Functions")
# 
import os
import numpy as np
import pandas as pd
import rasterio as rio
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
# Import the custom python script files
import my_Variogram
import my_RomanMetrics
import my_Variogram_Plot

### Manual Input
Please check these constants before running the code!

In [11]:
# DISC\\Python
cwd = "..\\..\\"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [12]:
# Set g
g = 4.66
# Spatial homogeneity ROIs
distances = [30,100,300,600,900]
# Spatial representativeness ROIs
# Set internal distance
distances_Int = [30,30,30,100,100,100,300,300]
# Set external distance
distances_Ext = [100,600,900,300,600,900,600,900]

### Read site info

In [13]:
# Find the coordinate based on the name of the site in our .csv file
df_Site = pd.read_excel(cwd + "//Site - With Moved ROI (Coordinates Only).xlsx")
df_Site.head()

,Number,Site,Latitude,Longitude,Reference network
0,1,ATGE,52.466778,12.959778,HYPERNET
1,2,ATLAS-Mohammed V,33.406152,-5.103319,Other
2,3,ATLAS-Mohammed V New,33.404814,-5.101614,Other
3,4,AT-Mmg,47.316700,10.970300,FLOX
4,5,BASP,39.049139,-2.075917,HYPERNET


### Real work begins here

In [14]:
for index in range(df_Site.shape[0]):
    
    temp_Site_Name = df_Site['Site'][index]
    print(f"{temp_Site_Name} starts!")
    if temp_Site_Name[-3:] == "New":
        list_std = []
        list_cv = []
        list_range = []
        list_nugget = []
        list_sill = []
        list_bins = []
        list_experimental_model = []
        list_fitted_model = []
        list_empiral_X = []
        list_empiral_Y = []
        for i in range(len(distances)):
            img = rio.open(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv " + str(distances[i]) + ".tif")  
            band = img.read(1).reshape(-1)
            temp_std = my_Variogram.calc_STD(band)
            temp_cv = my_Variogram.calc_CV(band)
            coordinates = my_Variogram.get_Coordinates_from_Rio(img)
            temp_range, temp_nugget, temp_sill, temp_bins, temp_experimental_model, temp_fitted_model, temp_empiral_X, temp_empiral_Y = my_Variogram.calc_Variogram_Parameters(band,coordinates,distances[i])
            list_std.append(temp_std)
            list_cv.append(temp_cv)
            list_range.append(temp_range)
            list_nugget.append(temp_nugget)
            list_sill.append(temp_sill)
            list_bins.append(temp_bins)
            list_experimental_model.append(temp_experimental_model)
            list_fitted_model.append(temp_fitted_model)
            list_empiral_X.append(temp_empiral_X)
            list_empiral_Y.append(temp_empiral_Y)
            # print(f"Distance {list_distances[i]} has been calculated!")

        list_Rcv = []
        list_RAW_Score = []
        list_Rse = []
        list_Rst = []
        list_Rsv = []
        list_ST_Score = []
        for i in range(len(distances)):
            dist_Int = distances_Int[i]
            dist_Ext = distances_Ext[i]
            index_Int = distances.index(dist_Int)
            index_Ext = distances.index(dist_Ext)
            range_Int, nugget_Int, sill_Int, bins_Int, experimental_model_Int, fitted_model_Int = list_range[index_Int], list_nugget[index_Int], list_sill[index_Int], list_bins[index_Int], list_experimental_model[index_Int], list_fitted_model[index_Int]
            range_Ext, nugget_Ext, sill_Ext, bins_Ext, experimental_model_Ext, fitted_model_Ext = list_range[index_Ext], list_nugget[index_Ext], list_sill[index_Ext], list_bins[index_Ext], list_experimental_model[index_Ext], list_fitted_model[index_Ext]
            cv_Int = list_cv[index_Int]
            cv_Ext = list_cv[index_Ext]
            semivar_range_Exp_Int = my_Variogram.get_Semivar_Exp(dist_Int, bins_Int, experimental_model_Int)
            semivar_range_Exp_Ext = my_Variogram.get_Semivar_Exp(dist_Ext, bins_Ext, experimental_model_Ext)
            # Calculate Rcv
            Rcv = my_RomanMetrics.calc_Rcv(cv_Ext, cv_Int)
            # Calculate RAW Score
            RAW_Score = my_RomanMetrics.calc_RAWScore(Rcv)
            # Calculate Rse
            Rse = my_RomanMetrics.calc_Rse(g, range_Ext, range_Int)
            # Calculate Rst
            Rst = my_RomanMetrics.calc_Rst(my_RomanMetrics.calc_ST(semivar_range_Exp_Ext, nugget_Ext), my_RomanMetrics.calc_ST(semivar_range_Exp_Int, nugget_Int))
            # Calculate Rsv
            Rsv = my_RomanMetrics.calc_Rsv(my_RomanMetrics.calc_SV(fitted_model_Ext, range_Ext, nugget_Ext, sill_Ext), my_RomanMetrics.calc_SV(fitted_model_Int, range_Int, nugget_Int, sill_Int))
            # Calculate ST Score
            ST_Score = my_RomanMetrics.calc_STScore(Rcv, Rse, Rst, Rsv)
            list_Rcv.append(Rcv)
            list_RAW_Score.append(RAW_Score)
            list_Rse.append(Rse)
            list_Rst.append(Rst)
            list_Rsv.append(Rsv)
            list_ST_Score.append(ST_Score)
            print(f"Internal distance {dist_Int} and external distance {dist_Ext} has been computed!")
        # Make a dataframe to illustrate the numeric result of variograms
        df_Vario = pd.DataFrame({
            "Distance": distances,
            "STD": list_std,
            "CV": list_cv,
            "Range": list_range,
            "Sill": list_sill,
            "Nugget": list_nugget
        })
        df_Vario
        # Make a dataframe to illustrate the numeric result of roman metrics
        df_Roman = pd.DataFrame({
            "Internal Distance": distances_Int,
            "External Distance": distances_Ext,
            "Rcv": list_Rcv,
            "Rse": list_Rse,
            "Rst": list_Rst,
            "Rsv": list_Rsv,
            "RAW Score": list_RAW_Score,
            "ST Score": list_ST_Score
        })
        df_Roman
        df_Vario.to_csv(cwd_Images_Processed + "\\" + temp_Site_Name  + "\\Vario.csv", index = False)
        df_Roman.to_csv(cwd_Images_Processed + "\\" + temp_Site_Name  + "\\Roman.csv", index = False)
        my_Variogram_Plot.plot_Variogram(distances, list_bins, list_fitted_model, list_range, list_sill, list_nugget, list_empiral_X,list_empiral_Y, site_Name_Graph = temp_Site_Name, path_SaveFig = cwd_Images_Processed + "\\" + temp_Site_Name  + "\\Variogram.png", bool_SaveFig = True, bool_CloseFig = True)
    print(f"{temp_Site_Name} done!")

ATGE starts!
ATGE done!
ATLAS-Mohammed V starts!
ATLAS-Mohammed V done!
ATLAS-Mohammed V New starts!


c:\ProgramData\anaconda3\envs\DISC\Lib\site-packages\skgstat\Variogram.py:1715: OptimizeWarning: Covariance of the parameters could not be estimated
  self.cof, self.cov = curve_fit(


Internal distance 30 and external distance 100 has been computed!
Internal distance 30 and external distance 300 has been computed!
Internal distance 100 and external distance 300 has been computed!
Internal distance 300 and external distance 600 has been computed!
Internal distance 300 and external distance 900 has been computed!
ATLAS-Mohammed V New done!
AT-Mmg starts!
AT-Mmg done!
BASP starts!
BASP done!
BE-Bra starts!
BE-Bra done!
BE-Maa starts!
BE-Maa done!
BE-Vie starts!
BE-Vie done!
BE-WAL starts!
BE-WAL done!
CANADA-ONT starts!
CANADA-ONT done!
CD-Ygb starts!
CD-Ygb done!
CHINA-AGR 1 starts!
CHINA-AGR 1 done!
CHINA-AGR 2 starts!
CHINA-AGR 2 done!
CHINA-FOR 1 starts!
CHINA-FOR 1 done!
CH-Lae starts!
CH-Lae done!
CZ-Lnz starts!
CZ-Lnz done!
Dahara (Senegal) starts!
Dahara (Senegal) done!
DEGE starts!
DEGE done!
DE-Geb starts!
DE-Geb done!
DE-Geb New starts!
Internal distance 30 and external distance 100 has been computed!
Internal distance 30 and external distance 300 has been c

In [16]:
# Concatenate .csv results of Roman Metrics
for i in range(df_Site.shape[0]):
    if os.path.exists(cwd_Images_Processed + "\\" + df_Site['Site'][i] + "\\Vario.csv"):
        temp_csv = pd.read_csv(cwd_Images_Processed + "\\" + df_Site['Site'][i] + "\\Vario.csv")
        temp_csv["Site"] = df_Site['Site'][i]
        if i == 0:
            final_csv_Variogram = temp_csv
        else:
            final_csv_Variogram = pd.concat((final_csv_Variogram,temp_csv), ignore_index= True)
final_csv_Variogram = final_csv_Variogram[["Site","Distance","STD","CV","Range","Sill","Nugget"]]
final_csv_Variogram.head(10)

,Site,Distance,STD,CV,Range,Sill,Nugget
0,ATGE,30,10.292980,0.256794,20.000000,71.762211,3.570080e+01
1,ATGE,100,15.866641,0.478179,70.000000,267.887789,5.104219e-14
2,ATGE,300,20.748732,0.428943,104.664412,449.836840,6.333086e+01
3,ATGE,600,21.184529,0.505233,227.522937,453.765630,9.937431e+01
4,ATGE,900,20.035755,0.502820,345.097181,438.814722,1.064037e+02
5,ATLAS-Mohammed V,30,1.490574,0.055660,20.000000,1.687615,8.395671e-01
6,ATLAS-Mohammed V,100,4.409100,0.170407,30.284278,18.800881,1.834641e-11
7,ATLAS-Mohammed V,300,6.381924,0.236488,98.388818,39.292309,1.650852e+01
8,ATLAS-Mohammed V,600,9.765243,0.368487,420.000000,89.614443,1.804581e+01
9,ATLAS-Mohammed V,900,11.160982,0.461659,630.000000,131.031057,1.541889e+00


In [18]:
final_csv_Variogram.to_csv(cwd + "\\Results\\Variogram.csv", index = False)

In [19]:
# Concatenate .csv results of Roman Metrics
for i in range(df_Site.shape[0]):
    if os.path.exists(cwd_Images_Processed + "\\" + df_Site['Site'][i] + "\\Roman.csv"):
        temp_csv = pd.read_csv(cwd_Images_Processed + "\\" + df_Site['Site'][i] + "\\Roman.csv")
        temp_csv["Site"] = df_Site['Site'][i]
        if i == 0:
            final_csv_Roman = temp_csv
        else:
            final_csv_Roman = pd.concat((final_csv_Roman,temp_csv), ignore_index= True)
final_csv_Roman = final_csv_Roman[["Site","Internal Distance","External Distance","Rcv","Rse","Rst","Rsv","RAW Score","ST Score"]]
final_csv_Roman.head(10)

,Site,Internal Distance,External Distance,Rcv,Rse,Rst,Rsv,RAW Score,ST Score
0,ATGE,30,100,0.862114,0.784802,0.990000,5.965000,0.579970,0.294941
1,ATGE,30,300,0.670379,0.788821,0.694206,7.947945,0.745846,0.256871
2,ATGE,100,300,-0.102966,0.923035,-0.148640,0.284701,4.855959,0.907602
3,ATGE,300,600,0.177857,0.952173,-0.045498,0.975952,2.811250,0.739677
4,ATGE,300,900,0.172231,0.954540,-0.119004,1.906935,2.903070,0.592676
5,ATLAS-Mohammed V,30,100,2.061553,0.756370,0.990000,2.013286,0.242536,0.409057
6,ATLAS-Mohammed V,30,300,3.248775,0.788388,0.223940,4.676587,0.153904,0.285321
7,ATLAS-Mohammed V,100,300,0.387784,0.851292,-0.384955,0.883853,1.289376,0.712510
8,ATLAS-Mohammed V,300,600,0.558165,0.952519,0.345489,4.879356,0.895793,0.347199
9,ATLAS-Mohammed V,300,900,0.952147,0.953194,0.610914,9.912784,0.525129,0.209272


In [20]:
final_csv_Roman.to_csv(cwd + "\\Results\\Roman.csv", index = False)